In [45]:
# Importación de las clases necesarias desde las bibliotecas
from azure.cosmos import CosmosClient, exceptions, PartitionKey
from datetime import datetime
import requests

# Definición de una función para formatear los datos de seguridad obtenidos de la API de BinaryEdge
def secDataFormat(ip, item_id):
    # Definición de los encabezados para la solicitud a la API de BinaryEdge
    headers = {'X-Key': 'tu_clave_aqui'}
    # Realización de una solicitud GET a la API de BinaryEdge para obtener datos de seguridad de la IP proporcionada
    data = requests.get(f"https://api.binaryedge.io/v2/query/ip/{ip}", headers=headers)

    try:
        # Intento de analizar la respuesta JSON de la API de BinaryEdge
        consume = data.json()
        # Extracción de los eventos de la respuesta JSON
        events = consume.get('events', [])
        # Inicialización de conjuntos para almacenar los estados y tipos de vulnerabilidad únicos
        status_set = set()
        type_set = set()

        # Iteración sobre los eventos devueltos por la API de BinaryEdge
        for event in events:
            # Extracción de los resultados de puertos para cada evento
            port_results = event.get('results', [])

            for result in port_results:
                # Extracción del estado de vulnerabilidad y tipo de vulnerabilidad de cada resultado
                status = result.get('result', {}).get('data', {}).get('status', 'No vulnerable')
                vulnerability_type = result.get('origin', {}).get('type', 'Sin BlueKeep')

                # Agregado del estado y tipo de vulnerabilidad a conjuntos para eliminar duplicados
                status_set.add(status)
                type_set.add(vulnerability_type)

        # Determinación del estado general de vulnerabilidad basado en los estados de vulnerabilidad
        overall_status = 'VULNERABLE' if 'VULNERABLE' in status_set else 'No vulnerable'

        # Formateo de los datos en un diccionario con el ID del ítem, IP, fecha, estado general y tipos de vulnerabilidad
        formatted_data = {
            'id': item_id,
            'partitionKey': ip,
            'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'status': [overall_status],
            'type': list(type_set)
        }

        # Devolución de los datos formateados
        return formatted_data
    
    except Exception as e:
        # Manejo de excepciones en caso de errores al procesar los datos de BinaryEdge
        print(f"Error al procesar los datos de BinaryEdge: {e}")

# Definición de una función para crear un contenedor en Azure Cosmos DB para los datos de una IP dada
def createIpContainer(ip, client, database_id):
    # Definición de las credenciales y configuración de Azure Cosmos DB... Se deben definir antes de ejecutar
    
    
    # Generación de un ID único para el contenedor basado en la IP y la fecha actual
    datetime_now = datetime.now().strftime('%Y%m%d%H%M%S')
    CONTAINER_ID = f"{ip.replace('.', '_')}-{datetime_now}"

    # Obtención de los datos de seguridad formateados utilizando la función secDataFormat
    ton_data = secDataFormat(ip, CONTAINER_ID)

    try:
        # Creación de un cliente de Cosmos DB
        client = CosmosClient(ENDPOINT, PRIMARY_KEY)
        # Obtención de la referencia a la base de datos
        database = client.get_database_client(DATABASE_ID)
        # Creación de un nuevo contenedor en la base de datos
        container = database.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
        # Impresión de un mensaje indicando que se ha creado el contenedor
        print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        try:
            # Inserción de los datos de seguridad en el contenedor
            container.upsert_item(body=ton_data)
            # Impresión de un mensaje indicando que los datos se han ingresado con éxito
            print(f'Datos de la IP {ip} ingresados con éxito en el contenedor {CONTAINER_ID}')

        except exceptions.CosmosHttpResponseError as e:
            # Manejo de excepciones en caso de errores al insertar el ítem en el contenedor
            print(f"Error de Cosmos DB al insertar el ítem: {e}")
        except Exception as e:
            # Manejo de excepciones en caso de errores inesperados al insertar el ítem
            print(f"Error inesperado al insertar el ítem: {e}")

    except exceptions.CosmosResourceExistsError:
        # Manejo de excepciones en caso de que el contenedor ya exista en la base de datos
        container = database.get_container_client(CONTAINER_ID)
        # Impresión de un mensaje indicando que se encontró el contenedor existente
        print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

# Llamada a la función createIpContainer para crear un contenedor para una IP específica
createIpContainer('80.120.238.18', client, DATABASE_ID)

Container with id '80_120_238_18-20240306141354' created
Datos de la IP 80.120.238.18 ingresados con éxito en el contenedor 80_120_238_18-20240306141354
